#### uploading data to google cloud using tool gsutil

In [1]:
!gsutil -m cp -r data/pq/ gs://snehil-devbucket/pq

Copying file://data/pq/green/2020/01/.part-00002-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/.part-00000-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/.part-00001-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/.part-00003-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/part-00000-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2020/01/part-00001-8fb612b7-68ba-4a94-98c3-d0fd1952b594-c000.snappy.parquet [Content-Type=application/octet-stream]..

* To connect to google cloud storage using hadoop we will need a connector(https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage#clusters)

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [3]:
credentials_location = '/home/snehil/gcloud.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/snehil/snehi/Downloads/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [4]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

your 131072x1 screen size is bogus. expect trouble
23/04/25 12:13:47 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.37.32 instead (on interface eth0)
23/04/25 12:13:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/25 12:13:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df_green = spark.read.parquet("gs://snehil-devbucket/pq/green/*/*")

In [7]:
df_green.count()

2722972

In [8]:
df_yellow = spark.read.parquet("gs://snehil-devbucket/pq/yellow/*/*")

In [9]:
df_yellow.count()

24648499